In [1]:
import os
from PyPDF2 import PdfReader
from typing import List, Tuple, Dict, Any
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA
from datasets import load_dataset
from langchain.schema import Document
import numpy as np

# DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

In [2]:
from llama_cpp import Llama
from langchain.embeddings.base import Embeddings

# 自定义 LangChain 的 Embeddings 类封装
class LlamaCppEmbeddings(Embeddings):
    def __init__(self, model_path: str):
        self.llm = Llama(model_path=model_path, embedding=True)

    def embed_documents(self, texts):
        # return [self.llm.embed(text)["data"][0]["embedding"] for text in texts]
        embeddings = []
        for text in texts:
            result = self.llm.embed(text)
            if isinstance(result, list) and isinstance(result[0], list):
                embeddings.append(result[0])
            else:
                embeddings.append(result)
        return embeddings

    def embed_query(self, text):
        # return self.llm.embed(text)["data"][0]["embedding"]
        result = self.llm.embed(text)
        return result[0] if isinstance(result, list) and isinstance(result[0], list) else result

In [3]:

class Proof:
    """
    隐私证明数据结构
    """
    def __init__(self, doc_id: str, score: float, vector: List[float], proof_data: Any):
        self.doc_id = doc_id
        self.score = score
        self.vector = vector
        self.proof_data = proof_data

In [ ]:
class Client:
    """
    轻量级rag客户端，负责数据集加载、向量存储构建与检索。
    """
    def __init__(self, model_path: str = "./models/Qwen3-Embedding/Qwen3-Embedding-0.6B-Q8_0.gguf", 
                vectorstore_path: str = "faiss_db"): # dashscope_api_key: str,使用api调用embedding模型
        os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")
        self.vectorstore_path = vectorstore_path
        # self.embeddings = DashScopeEmbeddings(
        #     model="text-embedding-v1",
        #     dashscope_api_key=dashscope_api_key
        # )
        self.embeddings = LlamaCppEmbeddings(model_path=model_path)
        self.db: FAISS = None

    def _chunk_text(self, text: str, chunk_size=1000, overlap= 200) -> list[str]:
        """
        将文本分块处理，使用递归字符分割器。
        """
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=overlap
        )
        return splitter.split_text(text)

    def _read_pdfs(self, pdf_paths: List[str]) -> str:
        text = []
        for path in pdf_paths:
            reader = PdfReader(path)
            for page in reader.pages:
                text.append(page.extract_text() or "")
        return "\n".join(text)

    def build_vectorstore_from_pdf(self, pdf_paths: list[str]) -> None:
        """
        处理pdf文件并构建FAISS向量存储。
        """
        raw = self._read_pdfs(pdf_paths)
        chunks = self._chunk_text(raw)
        self.db = FAISS.from_texts(chunks, embedding=self.embeddings)
        self.db.save_local(self.vectorstore_path)
        print(f"Vectorstore built at '{self.vectorstore_path}' with {len(chunks)} chunks.")

    def build_vectorstore_from_wiki(self, sample_size=100, batch_size=10):
        # 启用streaming模式在线读取huggingface datasets
        dataset = load_dataset("wikimedia/wikipedia", "20231101.en", streaming=True)
        iterator = iter(dataset["train"])
        texts = []
        count = 0
        for item in iterator:
            if count >= sample_size:
                break
            text = item.get("text", "")
            if text:
                texts.append(text)
                count += 1
        print(f"Total collected Wikipedia texts: {len(texts)}")

        # 分块并批量处理
        all_chunks = []
        for i, text in enumerate(texts):
            chunks = self._chunk_text(text)
            all_chunks.extend(chunks)

            # 每 batch_size 保存一次，防止内存溢出
            if len(all_chunks) >= batch_size or i == len(texts) - 1:
                if self.db is None:
                    self.db = FAISS.from_texts(all_chunks, embedding=self.embeddings)
                else:
                    self.db.add_texts(all_chunks)
                all_chunks.clear()
                print(f"Processed {i+1}/{len(texts)} articles...")

        # 保存向量库
        if self.db:
            self.db.save_local(self.vectorstore_path)
            print(f"Vectorstore saved to {self.vectorstore_path}")
        else:
            print("No data processed.")

    def load_vectorstore(self) -> None:
        """
        加载已保存的向量存储，并初始化检索器。
        """
        if not os.path.exists(self.vectorstore_path):
            raise FileNotFoundError(f"Vectorstore directory '{self.vectorstore_path}' not found.")
        self.db = FAISS.load_local(
            self.vectorstore_path,
            embeddings=self.embeddings,
            allow_dangerous_deserialization=True
        )
        self.retriever = self.db.as_retriever()
        print("Vectorstore loaded and retriever initialized.")

    def retrieve(self, query: str, top_k: int = 5) -> Tuple[List[Document], List[List[float]]]:
        """
        对输入 query 执行检索，返回 top_k 最相似文档及其向量。
        Output:
          docs: List[langchain.schema.Document]
          vectors: List[List[float]]
        """
        if self.db is None:
            raise RuntimeError("Vectorstore 未初始化，调用 load_vectorstore 或 build 方法先初始化。")
        # 生成 query 向量
        q_vec = self.embeddings.embed_query(query)
        print(q_vec)
        # 确保转换成 2D NumPy 数组，FAISS 要求 shape = (n_queries, dim)
        if isinstance(q_vec, list):
            q_vec = np.array(q_vec, dtype="float32").reshape(1, -1)
        # 使用 FAISS 原生 index.search
        D, I = self.db.index.search(q_vec, top_k)  # D: 距离, I: 索引ID
        docs = []
        vecs = []

        for idx in I[0]:
            if idx == -1:
                continue
            # 用 LangChain 的 docstore 获取 Document
            doc_id = self.db.index_to_docstore_id[idx]
            doc = self.db.docstore.search(doc_id)
            docs.append(doc)

            # 从 FAISS 中 reconstruct 向量
            vec = self.db.index.reconstruct(int(idx))
            vecs.append(vec)

        return docs, vecs
        # results = self.db.index.search(q_vec, top_k)
        # print(results)
        # ids, scores = results[1].tolist()[0], results[0].tolist()[0]
        # docs = [self.db.docstore.search(id) for id in ids]
        # vecs = [self.db.index.reconstruct(id) for id in ids]
        # return docs, vecs


    def query(self, question: str, top_k: int = 5) -> Tuple[List[Proof], List[float]]:
        """
        基于检索结果计算 Proof，并返回 proofs 列表及 query 向量。
        """
        docs, vecs = self.retrieve(question, top_k)
        proofs: List[Proof] = []
        for doc, vec in zip(docs, vecs):
            proof_data = {"doc_id": doc.metadata["source"], "merkle_path": []}
            proofs.append(Proof(doc.metadata.get("source", ""), score=0.0, vector=vec, proof_data=proof_data))
        q_vec = self.embeddings.embed_query(question)
        return proofs, q_vec
    
    # 这一段query包含了调用llm生成答案部分，一种是调用ollama部署的llm，一种是调用api并使用agent工具
    # def query(self, question: str) -> str:
    #     """
    #     基于已加载的向量存储进行查询，并返回生成的答案。
    #     """
    #     if self.retriever is None:
    #         raise RuntimeError("Retriever not initialized. Call 'load_vectorstore()' first.")
    #     llm = ChatOllama(model=self.ollama_model)
    #     qa_chain = RetrievalQA.from_chain_type(
    #         llm=llm,
    #         retriever=self.retriever,
    #         chain_type="stuff",
    #         return_source_documents=False
    #     )
    #     result = qa_chain.invoke({"query": question})
    #     return result["result"]
        # 创建检索工具
        # retrieval_tool = create_retriever_tool(
        #     self.retriever,
        #     name="pdf_extractor",
        #     description="Tool to answer queries based on the processed PDF content."
        # )

        # prompt = ChatPromptTemplate.from_messages([
        #     ("system", 
        #      """
        #      你是AI助手，请根据提供的上下文回答问题，确保提供所有细节，
        #      如果答案不在上下文中，请说 '答案不在上下文中'，不要提供错误的答案
        #      """),
        #     ("human", "{input}"),
        #     ("placeholder", "{agent_scratchpad}")
        # ])
        # agent = create_tool_calling_agent(llm, [retrieval_tool], prompt)
        # executor = AgentExecutor(agent=agent, tools=[retrieval_tool], verbose=False)
        # result = executor.invoke({"input": question})
        # return result.get('output', '')


In [ ]:
# 初始化 RAG 客户端
client = Client()
# 构建或加载向量库
pdf_files = ['D:\RAGnet\paper.pdf']
client.build_vectorstore_from_pdf(pdf_files)
client.load_vectorstore()
docs, vecs = client.retrieve("What is the main topic of the paper?")

llama_model_loader: loaded meta data with 36 key-value pairs and 310 tensors from ./models/Qwen3-Embedding/Qwen3-Embedding-0.6B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen3 Embedding 0.6b
llama_model_loader: - kv   3:                           general.basename str              = qwen3-embedding
llama_model_loader: - kv   4:                         general.size_label str              = 0.6B
llama_model_loader: - kv   5:                            general.license str              = apache-2.0
llama_model_loader: - kv   6:                   general.base_model.count u32              = 1
llama_model_loader: - kv  

KeyboardInterrupt: 

In [5]:
client = Client()
client.build_vectorstore_from_wiki(sample_size=100, batch_size=10)

llama_model_loader: loaded meta data with 36 key-value pairs and 310 tensors from ./models/Qwen3-Embedding/Qwen3-Embedding-0.6B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen3 Embedding 0.6b
llama_model_loader: - kv   3:                           general.basename str              = qwen3-embedding
llama_model_loader: - kv   4:                         general.size_label str              = 0.6B
llama_model_loader: - kv   5:                            general.license str              = apache-2.0
llama_model_loader: - kv   6:                   general.base_model.count u32              = 1
llama_model_loader: - kv  

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Total collected Wikipedia texts: 100


llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =     534.01 ms /   100 tokens (    5.34 ms per token,   187.26 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     542.37 ms /   101 tokens
llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =    1063.42 ms /   200 tokens (    5.32 ms per token,   188.07 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1077.61 ms /   201 tokens
llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =     244.87 ms /    46 tokens (    5.32 ms per token,   187.85 tokens per second)
llama_perf_context_print:        eval time = 

Processed 1/100 articles...


...

Processed 98/100 articles...


llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =     932.98 ms /   175 tokens (    5.33 ms per token,   187.57 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     941.74 ms /   176 tokens
llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =     958.11 ms /   181 tokens (    5.29 ms per token,   188.91 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     966.49 ms /   182 tokens
llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =     933.33 ms /   178 tokens (    5.24 ms per token,   190.71 tokens per second)
llama_perf_context_print:        eval time = 

Processed 99/100 articles...


llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =    1107.17 ms /   208 tokens (    5.32 ms per token,   187.87 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1116.86 ms /   209 tokens
llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =    1288.92 ms /   231 tokens (    5.58 ms per token,   179.22 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1297.92 ms /   232 tokens
llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =     697.45 ms /   126 tokens (    5.54 ms per token,   180.66 tokens per second)
llama_perf_context_print:        eval time = 

Processed 100/100 articles...
Vectorstore saved to faiss_db


In [6]:
client.load_vectorstore()

Vectorstore loaded and retriever initialized.


In [7]:
docs, vecs = client.retrieve("What is Abkhaz alphabet?", top_k=5)

llama_perf_context_print:        load time =     539.69 ms
llama_perf_context_print: prompt eval time =      88.81 ms /     8 tokens (   11.10 ms per token,    90.08 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      97.99 ms /     9 tokens


[3.9952266216278076, -16.86469268798828, 0.44079113006591797, -12.166587829589844, 2.918513774871826, -1.7121270895004272, 1.0849204063415527, -50.32857894897461, 1.9396106004714966, 6.704031467437744, -2.515451192855835, -1.750204086303711, 15.22193431854248, 0.6257795691490173, -7.34955358505249, 2.6439383029937744, -12.371712684631348, 4.164959907531738, -17.326148986816406, 3.081815481185913, -0.8964845538139343, -0.3004414737224579, -3.47270131111145, -6.245206832885742, -0.5454325675964355, -1.021567940711975, -2.2698965072631836, -15.29393196105957, -0.0789736956357956, -2.1437089443206787, 6.799513816833496, 5.4126877784729, -11.917704582214355, 1.8241513967514038, 0.07318972796201706, 0.7108349204063416, -1.1951684951782227, -2.7781360149383545, -3.081969976425171, 0.49996235966682434, 2.287668466567993, -0.5753247141838074, 1.0012001991271973, -1.489026665687561, -2.2935118675231934, -7.600603103637695, 2.375309944152832, 2.7748098373413086, 1.7590562105178833, 2.028740644454